In [2]:
pip install sentence-transformers faiss-cpu transformers


    click (>=7.0<=8.1.*)
          ~~~~~~^

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


    click (>=7.0<=8.1.*)
          ~~~~~~^

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import ollama
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd


c:\Users\Emna Nkhili\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Charger les données CSV
data = pd.read_csv('C:/Users/Emna Nkhili/Desktop/RAG-Project/data/DataExtaction.csv')


In [6]:
# Initialiser le modèle d'embedder
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Générer les embeddings
sentences = data['phrase'].tolist()  # Assurez-vous que 'phrase' existe dans le CSV
embeddings = embedder.encode(sentences, convert_to_tensor=False)


In [7]:
# Créer un index FAISS
embedding_dim = embeddings[0].shape[0]  # Dimension des embeddings
index = faiss.IndexFlatL2(embedding_dim)  # Créer un index FAISS avec distance L2
index.add(np.array(embeddings))  # Ajouter les embeddings à l'index


In [8]:
# Vérification de la connexion à Ollama
print("Vérification de la connexion à Ollama...")
try:
    ollama.chat(model="llama3", messages=[{'role': 'user', 'content': 'Test connexion'}])
    print("Connexion réussie à Ollama ✅")
except Exception as e:
    print("Erreur de connexion à Ollama ❌")
    print(e)
    raise SystemExit("Arrêt du programme.")


Vérification de la connexion à Ollama...
Connexion réussie à Ollama ✅


In [9]:
# Définir le modèle à utiliser
model_name = "llama3"

# Fonction pour générer la réponse
def generate_answer(user_query, top_k=5):
    # Embedding de la requête de l'utilisateur
    query_embedding = embedder.encode([user_query], convert_to_tensor=False)

    # Recherche dans FAISS
    distances, indices = index.search(np.array(query_embedding), top_k)
    matched_phrases = [sentences[idx] for idx in indices[0]]

    # Construire le contexte
    context = "\n".join(matched_phrases)

    # Construire le prompt
    full_prompt = f"""Tu es un expert en remèdes naturels.
Voici des exemples :
{context}

En te basant sur ces informations, réponds à la question suivante :
Question: {user_query}
"""

    # Envoyer le prompt à Ollama
    response = ollama.chat(
        model=model_name,
        messages=[{'role': 'user', 'content': full_prompt}]
    )

    return response['message']['content']


In [10]:
# Exemple d'utilisation
if __name__ == "__main__":
    question = ("Mon chien, un Labrador de 5 ans, montre des signes de léthargie, "
                "une perte d'appétit, et il vomit fréquemment. Après avoir consulté "
                "un vétérinaire, on lui a diagnostiqué une gastrite. Quels remèdes naturels "
                "puis-je utiliser pour soulager ses symptômes et améliorer son bien-être ?")
    
    answer = generate_answer(question)
    print("\n💬 Réponse générée par Ollama :\n")
    print(answer)



💬 Réponse générée par Ollama :

Je suis ravi de vous aider à trouver des remèdes naturels pour votre chien, un Labrador de 5 ans.

En résumé, les symptômes que votre chien présente (léthargie, perte d'appétit et vomissements fréquents) sont liés à une gastrite. Voici quelques suggestions de remèdes naturels qui pourraient aider à soulager ses symptômes et améliorer son bien-être :

1. **Gastrite** : Pour soulager les douleurs abdominales et réduire l'inflammation, vous pouvez essayer d'administrer ¼ de cuiller d'huile avec du miel cru ou de Manuka (ou un autre équivalent pour vos chats) deux fois par jour. Cela pourrait aider à apaiser la gorge et à favoriser une digestion plus saine.
2. **Digestion** : La nigelle, comme mentionné précédemment, a des propriétés détoxifiantes, digestives, diurétiques et carminatives. Elle peut aider à réguler le transit intestinal, ce qui est important pour votre chien qui présente une constipation ou diarrhée. Vous pouvez administrer 1 cuillère à café